In [1]:
import pandas as pd

df = pd.read_csv("Monthly PF Signals.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date')

# Identify the heuristic factor columns
factor_cols = [c for c in df.columns if c.startswith('r_')]

# Balanced panel: drop ANY NaN across all factor columns
F_balanced = df[factor_cols].dropna(axis=0, how='any')

print("Final sample:", F_balanced.index.min(), "→", F_balanced.index.max())
print("Months kept:", F_balanced.shape[0])
print("Factors used:", len(factor_cols))


Final sample: 1979-07-01 00:00:00 → 2019-12-01 00:00:00
Months kept: 480
Factors used: 55


/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_91315/3858089307.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


In [20]:
import numpy as np

# Convert to numpy for speed
Fmat = F_balanced.values        # shape T × K
mu_f = Fmat.mean(axis=0).reshape(-1, 1)   # K×1 mean vector
Sigma_f = np.cov(Fmat.T)                  # K×K covariance matrix

# Compute SDF weights (MV efficient portfolio inside factor span)
b_raw = np.linalg.inv(Sigma_f) @ mu_f
b = b_raw / np.linalg.norm(b_raw)         # normalize, direction matters only

# Compute heuristic SDF time series m_t = 1 - b' f_t
m = 1 - Fmat @ b
m_heuristic = pd.Series(m.flatten(), index=F_balanced.index, name="sdf_heuristic")

# Heuristic SDF Sharpe ratio
sr_heur = m_heuristic.mean() / m_heuristic.std()
print("Heuristic SDF Sharpe ratio:", sr_heur)


Heuristic SDF Sharpe ratio: 81.09512534043363


In [21]:
m_heuristic

date
1979-07-01    0.974190
1979-08-01    0.990296
1979-09-01    0.997862
1979-10-01    0.991755
1979-11-01    0.986192
                ...   
2019-08-01    1.011213
2019-09-01    0.976170
2019-10-01    0.997755
2019-11-01    0.994252
2019-12-01    1.009889
Name: sdf_heuristic, Length: 480, dtype: float64

In [27]:
import pandas as pd
import numpy as np

# Load MONTHLY stock-level signals
df_m = pd.read_csv("Stock level signals.csv")
df_m['date'] = pd.to_datetime(df_m['date'])
df_m = df_m.sort_values(['date', 'permno']).reset_index(drop=True)

# rename returns
df_m = df_m.rename(columns={'re': 'ret'})


/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/416621955.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_m['date'] = pd.to_datetime(df_m['date'])


In [28]:
meta_cols = ['permno', 'date', 'ret', 'price', 'age']
X_cols = [c for c in df_m.columns if c not in meta_cols]

print("Number of characteristics:", len(X_cols))


Number of characteristics: 52


In [ ]:
def rank_normalize(x):
    # If all missing → return zeros
    if x.isna().all():
        return x.fillna(0)

    # Rank (0,1) scaling
    r = rankdata(x, method='average') / (len(x) + 1)

    # Gaussianization
    z = norm.ppf(r)

    return pd.Series(z, index=x.index)

# Apply per month
df_m[X_cols] = df_m.groupby('date')[X_cols].transform(rank_normalize)


In [29]:
R_mat = df_m.pivot(index='permno', columns='date', values='ret').fillna(0)

# demean across time
R_dm = R_mat.subtract(R_mat.mean(axis=1), axis=0)


In [31]:
U, S, Vt = np.linalg.svd(R_dm.values, full_matrices=False)

K_pca = 40
B_pca = U[:, :K_pca]             # loadings: N × Kpca
Lambda_pca = np.diag(S[:K_pca])  # Kpca × Kpca


In [32]:
resid = R_dm.values - B_pca @ (B_pca.T @ R_dm.values)
D_idio = np.var(resid, axis=1)


In [33]:
months = sorted(df_m['date'].unique())

gls_list = []

for dt in months:

    tmp = df_m[df_m['date'] == dt]
    
    perm = tmp['permno'].values
    idx = [list(R_mat.index).index(p) for p in perm]

    X_t = tmp[X_cols].values                # N×K characteristics
    r_t = tmp['ret'].values.reshape(-1,1)   # N×1
    B_t = B_pca[idx, :]                     # N×Kpca
    D_t = D_idio[idx]                       # N-vector

    Dinv = np.diag(1.0 / D_t)

    mid = np.linalg.inv(np.linalg.inv(Lambda_pca) + B_t.T @ Dinv @ B_t)

    Sigma_inv = Dinv - Dinv @ B_t @ mid @ B_t.T @ Dinv

    XtSinv = X_t.T @ Sigma_inv
    M = np.linalg.inv(XtSinv @ X_t)

    f_gls_t = M @ XtSinv @ r_t

    gls_list.append((dt, f_gls_t.flatten()))


/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/1585143383.py:17: RuntimeWarning: divide by zero encountered in divide
  Dinv = np.diag(1.0 / D_t)
/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/1585143383.py:21: RuntimeWarning: invalid value encountered in matmul
  Sigma_inv = Dinv - Dinv @ B_t @ mid @ B_t.T @ Dinv
/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/1585143383.py:17: RuntimeWarning: divide by zero encountered in divide
  Dinv = np.diag(1.0 / D_t)
/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/1585143383.py:21: RuntimeWarning: invalid value encountered in matmul
  Sigma_inv = Dinv - Dinv @ B_t @ mid @ B_t.T @ Dinv
/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/1585143383.py:17: RuntimeWarning: divide by zero encountered in divide
  Dinv = np.diag(1.0 / D_t)
/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/1585143383.py:21: RuntimeWarning: invalid value encountered in matm

In [34]:
df_gls = pd.DataFrame(gls_list, columns=['date', 'gls_vec'])
df_gls['date'] = pd.to_datetime(df_gls['date'])

df_gls_exp = pd.DataFrame(df_gls['gls_vec'].tolist(), index=df_gls['date'])
df_gls_exp.index.name = 'date'


In [35]:
import numpy as np
import pandas as pd

# F_heur and F_gls must have the SAME dates and SAME columns
# Ensure aligned
F_heur = F_balanced.copy()        # your heuristic factor panel (monthly)
F_gls = df_gls_exp.copy()         # your GLS factor panel (monthly)

# Align dates
common = F_heur.index.intersection(F_gls.index)
Fh = F_heur.loc[common].values     # T × K
Fg = F_gls.loc[common].values      # T × K


In [36]:
# compute projection matrix P (K×K)
P = (Fh.T @ Fg) @ np.linalg.inv(Fg.T @ Fg)


In [37]:
F1 = Fh - (Fg @ P.T)
df_F1 = pd.DataFrame(F1, index=common, columns=F_heur.columns)


In [38]:
F2 = F1 - (Fg @ P.T)
df_F2 = pd.DataFrame(F2, index=common, columns=F_heur.columns)


In [43]:
# Align heuristic and GLS factors
common_idx = F_heur.index.intersection(F_gls.index)

Fh = F_heur.loc[common_idx].values
Fg = F_gls.loc[common_idx].values

# Save metadata for reconstruction
cols = F_heur.columns          # <-- this was missing
idx = common_idx


In [44]:
def hedge_factors(Fh, Fg, P, num_iter=2):
    """
    Iteratively hedge heuristic factors toward GLS using projection P.
    Fh: T × K matrix (heuristic)
    Fg: T × K matrix (GLS)
    P:  K × K projection matrix
    num_iter: number of hedging iterations
    """
    F_current = Fh.copy()

    for i in range(num_iter):
        # F_new = F_old - (Fgls @ Pᵀ)
        F_current = F_current - (Fg @ P.T)

    return F_current


In [59]:
print("Heuristic shape:", Fh.shape)
print("Heuristic NaNs:", np.isnan(Fh).sum())
print("Heuristic std:", np.std(Fh, axis=0))


Heuristic shape: (480, 55)
Heuristic NaNs: 0
Heuristic std: [0.04464105 0.04634092 0.0444393  0.04572609 0.04564788 0.04909924
 0.04628545 0.04658261 0.04965007 0.04326372 0.04608241 0.04213004
 0.04396685 0.04126625 0.04717719 0.04575954 0.04703503 0.04672462
 0.0495816  0.04810909 0.04746931 0.04785935 0.04356544 0.04622972
 0.04745154 0.03681338 0.04392307 0.0439377  0.04741363 0.04925691
 0.04628773 0.0469347  0.04101831 0.04776039 0.04722143 0.04492487
 0.04733838 0.05011269 0.0426262  0.04521815 0.04665339 0.04273117
 0.04788762 0.04769475 0.04683387 0.0452197  0.04638187 0.04192922
 0.0464823  0.04507837 0.04643892 0.04968729 0.04676341 0.04606432
 0.04310611]


In [45]:
F_hedge1 = hedge_factors(Fh, Fg, P, num_iter=1)
df_F1 = pd.DataFrame(F_hedge1, index=idx, columns=cols)


In [47]:
F_hedge2 = hedge_factors(Fh, Fg, P, num_iter=2)
df_F2 = pd.DataFrame(F_hedge2, index=idx, columns=cols)


In [46]:
F_hedge5 = hedge_factors(Fh, Fg, P, num_iter=5)
df_F5 = pd.DataFrame(F_hedge5, index=idx, columns=cols)


In [49]:
import numpy as np

def build_sdf(F):
    """
    F : T × K numpy array of factor returns
    Returns:
        m : SDF time series (T,)
        b : SDF weights (K,)
        mu : factor means (K,)
        Sigma : factor covariance (K×K)
    """
    # Mean vector (K×1)
    mu = F.mean(axis=0).reshape(-1,1)

    # Covariance matrix (K×K)
    Sigma = np.cov(F, rowvar=False)

    # Compute b = Σ^-1 μ
    Sigma_inv = np.linalg.inv(Sigma)
    b = Sigma_inv @ mu            # (K×1)

    # Normalize so pricing kernel has λ = 1
    scale = float(mu.T @ Sigma_inv @ mu)
    b = b / scale                 # (K×1)

    # SDF time series: m_t = 1 − b'F_t
    m = 1 - (F @ b).flatten()

    return m, b.flatten(), mu.flatten(), Sigma


In [51]:
# Heuristic
Fh_np = F_heur.loc[idx].values

# GLS
Fg_np = F_gls.loc[idx].values

# Hedged 1
F1_np = df_F1.loc[idx].values

# Hedged 2
F2_np = df_F2.loc[idx].values

# Hedged 5
F5_np = df_F5.loc[idx].values


In [52]:
m_heur, b_heur, mu_h, Sig_h = build_sdf(Fh_np)
m_gls,  b_gls,  mu_g, Sig_g = build_sdf(Fg_np)
m_h1,   b_h1,   mu1,   Sig1 = build_sdf(F1_np)
m_h2,   b_h2,   mu2,   Sig2 = build_sdf(F2_np)
m_h5,   b_h5,   mu5,   Sig5 = build_sdf(F5_np)


/var/folders/q1/c_66kq_d1xl8r_282r8p1mfw0000gn/T/ipykernel_75295/2552884305.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  scale = float(mu.T @ Sigma_inv @ mu)


In [54]:
def sdf_sharpe(m):
    return m.mean() / m.std()
def hj_distance(m):
    mu = m.mean()
    sd = m.std()
    return np.sqrt(mu*mu + sd*sd) - mu


In [55]:
SR_heur = sdf_sharpe(m_heur)
SR_gls  = sdf_sharpe(m_gls)
SR_h1   = sdf_sharpe(m_h1)
SR_h2   = sdf_sharpe(m_h2)
SR_h5   = sdf_sharpe(m_h5)

HJ_heur = hj_distance(m_heur)
HJ_gls  = hj_distance(m_gls)
HJ_h1   = hj_distance(m_h1)
HJ_h2   = hj_distance(m_h2)
HJ_h5   = hj_distance(m_h5)


In [56]:
comparison_table = pd.DataFrame({
    "Sharpe": [SR_heur, SR_h1, SR_h2, SR_h5, SR_gls],
    "HJ Distance": [HJ_heur, HJ_h1, HJ_h2, HJ_h5, HJ_gls]
}, index=["Heuristic", "Hedged1", "Hedged2", "Hedged5", "GLS"])

comparison_table


,Sharpe,HJ Distance
Heuristic,-2.825912e-16,1.152426
Hedged1,NaN,NaN
Hedged2,NaN,NaN
Hedged5,NaN,NaN
GLS,NaN,NaN
